In [5]:
from sympy import *
d0, d1, ani = symbols('d0, d1, ani')
eta_pwl, G  = symbols('eta_pwl, G')
ani_fact    = symbols('ani_fact')
nx, ny = symbols('nx, ny')
two = symbols('two')

# ani  = 1.0 - 1.0/ani_fact
Dani = Matrix([[2.0 - 2.0*ani*d0, 2.0*ani*d0, 2.0*ani*d1*two], 
[2.0*ani*d0, 2.0 - 2.0*ani*d0, -2.0*ani*d1*two],
[2.0*ani*d1, -2.0*ani*d1, two*(1+2.0*ani*(d0 - 0.5))]])

iDani  = Dani.inv()

Dv     = eta_pwl * Dani
 
iDv1   = Dv.inv()
iDv2   = 1.0/eta_pwl * iDani
iDe2   = 1.0/G       * iDani
iDve2  = (1.0/eta_pwl + 1.0/G) * iDani
Dve    = 1.0/(1.0/eta_pwl + 1.0/G) * Dani

# Dve1  = iDve2.inv()

In [6]:
# This shows the anisotropy matrix for MDOODZ6.0
# d0     = 2.0*nx**2.0*ny**2.0
# d1     = nx*ny*(-nx**2.0 + ny**2.0)
# Dv_MD6 = Dv.subs('d0', d0).subs('d1', d1)
Dv_MD6 = Dv
display( Dv_MD6 )


Matrix([
[eta_pwl*(-2.0*ani*d0 + 2.0),          2.0*ani*d0*eta_pwl,               2.0*ani*d1*eta_pwl*two],
[         2.0*ani*d0*eta_pwl, eta_pwl*(-2.0*ani*d0 + 2.0),              -2.0*ani*d1*eta_pwl*two],
[         2.0*ani*d1*eta_pwl,         -2.0*ani*d1*eta_pwl, eta_pwl*two*(2.0*ani*(d0 - 0.5) + 1)]])

In [7]:
# This is what is the StressEvolutionNonLinearAnisotropy_v9 script
# nx       = cos(angle); ny = sin(angle);
# two      = 1.0
Director = Matrix([ [nx], [ny]])
d0, d1 = symbols('d0, d1')
# d0       = 2.0*Director[0]**2.0*Director[1]**2.0
# d1       = Director[0]*Director[1] * (-Director[0]**2.0 + Director[1]**2.0)
C_ANI    = Matrix([ [-d0, d0, two*d1], [d0, -d0, -two*d1], [d1, -d1, -two*(1/2-d0)]]) ##### !!! - sign in D33 -a0
C_ISO    = Matrix([ [1, 0, 0], [0, 1, 0], [0, 0, two*1/2]])
Dani     = 2.0* (C_ISO + C_ANI * ani)
Dv_Matlab= eta_pwl * Dani
display( Dv_Matlab )

Matrix([
[eta_pwl*(-2.0*ani*d0 + 2.0),          2.0*ani*d0*eta_pwl,                      2.0*ani*d1*eta_pwl*two],
[         2.0*ani*d0*eta_pwl, eta_pwl*(-2.0*ani*d0 + 2.0),                     -2.0*ani*d1*eta_pwl*two],
[         2.0*ani*d1*eta_pwl,         -2.0*ani*d1*eta_pwl, eta_pwl*(-2.0*ani*two*(0.5 - d0) + 1.0*two)]])

In [8]:
# Now the difference between the 2 formulations
nerr = 0
for j in range(3):
    for i in range(3):
        if abs( (Dv_Matlab[j,i] - Dv_MD6[j,i]).simplify() ) > 0: 
            display( 'error' )
            nerr+=1
if nerr==0:
    display( 'everything is fine' )

'everything is fine'

In [40]:
# This is just have a look the Gij components
Exx,Ezz,Exz = symbols('Exx,Ezz,Exz')
E           = Matrix([[Exx],[Ezz],[Exz]])
Gij = (C_ISO + C_ANI * ani)*E
display(Gij)

Matrix([
[            Exx*(-ani*d0 + 1) + Exz*ani*d1*two + Ezz*ani*d0],
[            Exx*ani*d0 - Exz*ani*d1*two + Ezz*(-ani*d0 + 1)],
[Exx*ani*d1 + Exz*(-ani*two*(0.5 - d0) + two/2) - Ezz*ani*d1]])

In [27]:
Exx,Ezz,Exz = symbols('Exx,Ezz,Exz')
Txx,Tzz,Txz = symbols('Txx,Tzz,Txz')
eta_eff     = Function('eta_eff')(Exx,Ezz,Exz)
E           = Matrix([[Exx],[Ezz],[Exz]])
Dv_jac      = eta_eff * Dani
T           = Dv_jac * E

Gxx = Gij[0]
Gzz = Gij[1]
Gxz = Gij[2]

# First row
display(collect( T[0].diff(Exx), eta_eff.diff(Exx)).subs(Gxx,'Gxx').simplify().subs(eta_eff,'eta_eff') )
display(Gxx)



display(collect( T[2].diff(Exx), eta_eff.diff(Exx)).subs(Gxz,'Gxz').simplify().subs(eta_eff,'eta_eff') )
display(collect( T[2].diff(Ezz), eta_eff.diff(Ezz)).subs(Gxz,'Gxz').simplify().subs(eta_eff,'eta_eff') )
display(collect( T[2].diff(Exz), eta_eff.diff(Exz)).subs(Gxz,'Gxz').simplify().subs(eta_eff,'eta_eff') )
display(Gxx)
# display(T[1].diff(Exx).subs(eta_eff,'eta_eff'))
# display(T[2].diff(Exx).subs(eta_eff,'eta_eff'))

# # Second row
# display(T[0].diff(Ezz).subs(eta_eff,'eta_eff'))
# display(T[1].diff(Ezz).subs(eta_eff,'eta_eff'))
# display(T[2].diff(Ezz).subs(eta_eff,'eta_eff'))

# # Third row
# display(T[0].diff(Exz).subs(eta_eff,'eta_eff'))
# display(T[1].diff(Exz).subs(eta_eff,'eta_eff'))
# display(T[2].diff(Exz).subs(eta_eff,'eta_eff'))


-2.0*eta_eff*(ani*d0 - 1) + 2.0*(-Exx*(ani*d0 - 1) + Exz*ani*d1 + Ezz*ani*d0)*Derivative(eta_eff, Exx)

Exx*(-ani*d0 + 1) + 1.0*Exz*ani*d1 + Ezz*ani*d0

2.0*ani*d1*eta_eff + (2.0*Exx*ani*d1 + Exz*(2.0*ani*(1.0*d0 - 0.5) + 1.0) - 2.0*Ezz*ani*d1)*Derivative(eta_eff, Exx)

-2.0*ani*d1*eta_eff + (2.0*Exx*ani*d1 + Exz*(2.0*ani*(1.0*d0 - 0.5) + 1.0) - 2.0*Ezz*ani*d1)*Derivative(eta_eff, Ezz)

eta_eff*(2.0*ani*(1.0*d0 - 0.5) + 1.0) + (2.0*Exx*ani*d1 + Exz*(2.0*ani*(1.0*d0 - 0.5) + 1.0) - 2.0*Ezz*ani*d1)*Derivative(eta_eff, Exz)

Exx*(-ani*d0 + 1) + 1.0*Exz*ani*d1 + Ezz*ani*d0

In [7]:
Txx0, exx, Tyy0, eyy  = symbols('Txx0, exx, Tyy0, eyy')
etae, etave = symbols('etae, etave')
Exx  = exx + Txx0/2/etae
Txx  = 2*etave*Exx
exxe = (Txx -Txx0 )/2/etae
Eyy  = eyy + Tyy0/2/etae
Tyy  = 2*etave*Eyy
eyye = (Tyy -Tyy0 ) /etae
eiie = sqrt(1/2*(exxe**2 + eyye**2))
Tii  = sqrt(1/2*( Txx**2 +  Tyy**2))
Eii  = sqrt(1/2*( Exx**2 +  Eyy**2))
exxe1= eiie*Txx/Tii

display(exxe)
display(eyye)
display(Eii)
display( (exxe1-exxe).simplify())

(-Txx0/2 + etave*(Txx0/(2*etae) + exx))/etae

(-Tyy0 + 2*etave*(Tyy0/(2*etae) + eyy))/etae

0.707106781186548*sqrt((Txx0/(2*etae) + exx)**2 + (Tyy0/(2*etae) + eyy)**2)

(etae*etave*sqrt(((Txx0*etae - etave*(Txx0 + 2*etae*exx))**2 + 4*(Tyy0*etae - etave*(Tyy0 + 2*etae*eyy))**2)/etae**4)*(Txx0 + 2*etae*exx) + sqrt(etave**2*((Txx0 + 2*etae*exx)**2 + (Tyy0 + 2*etae*eyy)**2)/etae**2)*(Txx0*etae - etave*(Txx0 + 2*etae*exx)))/(2*etae**2*sqrt(etave**2*((Txx0 + 2*etae*exx)**2 + (Tyy0 + 2*etae*eyy)**2)/etae**2))

In [9]:
exx, eyy, exy = symbols('exx, eyy, exy')
eta = symbols('eta')
I2      =       1/2*(exx**2 + eyy**2) + exy**2
###################################
Txx = 2*eta*exx
Tyy = 2*eta*eyy
Txy = 2*eta*exy

eii = sqrt( 1/2*(exx**2 + eyy**2) + exy**2)
Tii = sqrt( 1/2*(Txx**2 + Tyy**2) + Txy**2)
eta_eff = Tii/2/eii
eta_eff = eta_eff.subs(I2,'I2')
display(  eta_eff )

################################################

d11,d22,d33,d12,d13,d23 = symbols('d11,d22,d33,d12,d13,d23')
Exx_ani = d11*exx + d12*eyy + d13*exy
Eyy_ani = d12*exx + d22*eyy + d23*exy
Exy_ani = d13*exx + d23*eyy + d33*exy
Txx     = 2* eta * Exx_ani
Tyy     = 2* eta * Eyy_ani
Txy     = 2* eta * Exy_ani

I2      =       1/2*(exx**2 + eyy**2) + exy**2
I2_ani  =       1/2*(Exx_ani**2 + Eyy_ani**2) + Exy_ani**2
eii = sqrt( I2 )
Tii = sqrt( 1/2*(Txx**2 + Tyy**2) + Txy**2)
eta_eff = Tii/eii/2
eta_eff = eta_eff.subs(I2,'I2').subs(I2_ani,'I2_ani')
display(  eta_eff.simplify() )





sqrt(0.5*eta**2*exx**2 + eta**2*exy**2 + 0.5*eta**2*eyy**2)/sqrt(I2)

sqrt(eta**2*(0.5*(d11*exx + d12*eyy + d13*exy)**2 + 0.5*(d12*exx + d22*eyy + d23*exy)**2 + (d13*exx + d23*eyy + d33*exy)**2))/sqrt(I2)

In [10]:
display(Txx/2/exx)

eta*(d11*exx + d12*eyy + d13*exy)/exx

In [19]:
C_pwl, npwl, eta_ve = symbols('C_pwl, npwl, eta_ve')
Gxx, Exx, Gyy, Gzz, Gxy, Eyy, Ezz, Exy, f_ani, Tii = symbols('Gxx, Exx, Gyy, Gzz, Gxy, Eyy, Ezz, Exy, f_ani, Tii')
# Tii     = 2*eta*Eii

Eii_pwl = C_pwl * Tii**(npwl) 
display(Eii_pwl.diff(eta).subs(Eii_pwl, 'Eii_pwl'))

# Dislocation work
exx_pwl = Exx/Gxx*f_ani * Txx/Tii * Eii_pwl
eyy_pwl = Eyy/Gyy*f_ani * Tyy/Tii * Eii_pwl
# exx_pwl = Txx/Tii * Eii_pwl
# eyy_pwl = Tyy/Tii * Eii_pwl
Wdis    = exx_pwl*Txx + eyy_pwl*Tyy
display(Wdis.subs(Eii_pwl, 'Eii_pwl').subs(Eii,'Eii').subs(exx_pwl,'exx_pwl').subs(Txx, 'Txx').subs(Tyy, 'Tyy').subs(Tii, 'Tii').simplify())
display(Eii)



0

4.0*Eii_pwl*eta_ve**2*(Exx*Gxx + Eyy*Gyy)/(Tii*f_ani)

0.707106781186548*sqrt((Txx0/(2*etae) + exx)**2 + (Tyy0/(2*etae) + eyy)**2)

In [23]:
d, p = symbols('d,p')

# Jacobian for coupled power-law and GSE
Tii       = 2*eta_ve*Eii
Txx       = Tii/f_ani/Eii*Gxx
Tyy       = Tii/f_ani/Eii*Gyy
Tzz       = -Txx-Tyy
Txy       = Tii/f_ani/Eii*Gxy


Wpwl      = f_ani*Eii_pwl/Tii*( Exx/Gxx*Txx**2 + Eyy/Gyy*Tyy**2 + Ezz/Gzz*Tzz**2 + 2*Exy/Gxy*Txy**2)

# Ared = lam/c/gam
# Agro = Kg*fH20*exp(-(Eg+P*Vg)/R/T)*p^(-1)
Ared, Agro   = symbols('Ared, Agro')
dred         =-Ared*Wpwl*d**2
dgro         = Agro*d**(1-p)